#### This is the first notebook for working through Real-World NLP

Hagiwara, Masato. Real-World Natural Language Processing (p. 83). Manning. Kindle Edition. 

Everything in the cell below ran in my python 3.9.7 poetry environment :-)

In [1]:
from itertools import chain
from typing import Dict
 
import numpy as np
import torch
import torch.optim as optim
from allennlp.data.data_loaders import MultiProcessDataLoader
from allennlp.data.samplers import BucketBatchSampler
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.nn.util import get_text_field_mask
from allennlp.training import GradientDescentTrainer
from allennlp.training.metrics import CategoricalAccuracy, F1Measure
from allennlp_models.classification.dataset_readers.stanford_sentiment_tree_bank import StanfordSentimentTreeBankDatasetReader

In [2]:
EMBEDDING_DIM = 128
HIDDEN_DIM = 128

In [3]:
reader = StanfordSentimentTreeBankDatasetReader()
train_path = 'https://s3.amazonaws.com/realworldnlpbook/data/stanfordSentimentTreebank/trees/train.txt'
dev_path = 'https://s3.amazonaws.com/realworldnlpbook/data/stanfordSentimentTreebank/trees/dev.txt'

That completes the initial setup.

#### 2.3.2 Using Word Embedding for snteiment analysis


Create dataset loaders to pass data to training pipeline

In [4]:
#!poetry run jupyter nbextension enable --py widgetsnbextension

Create dataset loaders

In [5]:
sampler = BucketBatchSampler(batch_size=32, sorting_keys=["tokens"])
train_data_loader = MultiProcessDataLoader(reader, train_path, batch_sampler=sampler)
dev_data_loader = MultiProcessDataLoader(reader, dev_path, batch_sampler = sampler)

loading instances: 0it [00:00, ?it/s]

loading instances: 0it [00:00, ?it/s]

Create a vocabularly instance from a set of instances

In [6]:
vocab = Vocabulary.from_instances(chain(train_data_loader.iter_instances(), dev_data_loader.iter_instances()), min_count={'tokens':3})

building vocab: 0it [00:00, ?it/s]

The size of embeddigns is governed by EMBEDDING_DIM

In [21]:
train_data_loader.index_with(vocab)
dev_data_loader.index_with(vocab)

In [7]:
token_embeddings = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),embedding_dim=EMBEDDING_DIM)

Specify which index names corrrespond to which bembeddigns and pass it to BasicTextFieldEmbedder

In [8]:
word_embeddings = BasicTextFieldEmbedder({'tokens': token_embeddings})

Create an RNN, specifically an LSTM (Long Short-Term Memory)

In [9]:
encoder = PytorchSeq2VecWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

Create a linear Layer

In [26]:
# Model in AllenNLP represents a model that is trained.
class LstmClassifier(Model):
    def __init__(self,
                 embedder: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 vocab: Vocabulary,
                 positive_label: str = '4') -> None:
        super().__init__(vocab)
        # We need the embeddings to convert word IDs to their vector representations
        self.embedder = embedder

        self.encoder = encoder

        # After converting a sequence of vectors to a single vector, we feed it into
        # a fully-connected linear layer to reduce the dimension to the total number of labels.
        self.linear = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))

        # Monitor the metrics - we use accuracy, as well as prec, rec, f1 for 4 (very positive)
        positive_index = vocab.get_token_index(positive_label, namespace='labels')
        self.accuracy = CategoricalAccuracy()
        self.f1_measure = F1Measure(positive_index)

        # We use the cross entropy loss because this is a classification task.
        # Note that PyTorch's CrossEntropyLoss combines softmax and log likelihood loss,
        # which makes it unnecessary to add a separate softmax layer.
        self.loss_function = torch.nn.CrossEntropyLoss()

    # Instances are fed to forward after batching.
    # Fields are passed through arguments with the same name.
    def forward(self,
                tokens: Dict[str, torch.Tensor],
                label: torch.Tensor = None) -> torch.Tensor:
        # In deep NLP, when sequences of tensors in different lengths are batched together,
        # shorter sequences get padded with zeros to make them equal length.
        # Masking is the process to ignore extra zeros added by padding
        mask = get_text_field_mask(tokens)

        # Forward pass
        embeddings = self.embedder(tokens)
        encoder_out = self.encoder(embeddings, mask)
        logits = self.linear(encoder_out)

        # In AllenNLP, the output of forward() is a dictionary.
        # Your output dictionary must contain a "loss" key for your model to be trained.
        output = {"logits": logits}
        if label is not None:
            self.accuracy(logits, label)
            self.f1_measure(logits, label)
            output["loss"] = self.loss_function(logits, label)

        return output

    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        return {'accuracy': self.accuracy.get_metric(reset),
                **self.f1_measure.get_metric(reset)}

In [27]:
optimizer = optim.Adam(model.parameters())

Define and initialise model

In [28]:
model = LstmClassifier(word_embeddings, encoder, vocab)

Allennlp  'Trainer' class acts as a framework for putting sll thr components together and managing the training pipeline

In [29]:
trainer = GradientDescentTrainer(
    model=model,
    optimizer=optimizer,
    data_loader=train_data_loader,
    validation_data_loader=dev_data_loader,
    patience=10,
    num_epochs=20,
    cuda_device=-1)
 
trainer.train()

  0%|          | 0/267 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/267 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/267 [00:00<?, ?it/s]

...

  0%|          | 0/267 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/267 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

{'best_epoch': 1,
 'peak_worker_0_memory_MB': 340.74609375,
 'training_duration': '0:01:44.733839',
 'epoch': 11,
 'training_accuracy': 0.9207631086142322,
 'training_precision': 0.9501187801361084,
 'training_recall': 0.9316770434379578,
 'training_f1': 0.9408075213432312,
 'training_loss': 0.2790430114510354,
 'training_worker_0_memory_MB': 340.74609375,
 'validation_accuracy': 0.3678474114441417,
 'validation_precision': 0.38372093439102173,
 'validation_recall': 0.4000000059604645,
 'validation_f1': 0.3916914165019989,
 'validation_loss': 2.2758769069399154,
 'best_validation_accuracy': 0.368755676657584,
 'best_validation_precision': 0.4204545319080353,
 'best_validation_recall': 0.4484848380088806,
 'best_validation_f1': 0.4340175986289978,
 'best_validation_loss': 1.3868441070829118}

Allennlp provides a framework called 'predictors' whose job it is to receive input in raw form and (e.g. a string) ,pass it througn pre-processing and the neural network pipeline and return the results.<br>
Here Hagiwara has written a predictor for SST called 'SentenceClassifierPredictor'

In [32]:
from allennlp.common import JsonDict
from allennlp.data import DatasetReader, Instance
from allennlp.data.tokenizers.spacy_tokenizer import SpacyTokenizer
from allennlp.models import Model
from allennlp.predictors import Predictor
from overrides import overrides
from typing import List


# You need to name your predictor and register so that `allennlp` command can recognize it
# Note that you need to use "@Predictor.register", not "@Model.register"!
@Predictor.register("sentence_classifier_predictor")
class SentenceClassifierPredictor(Predictor):
    def __init__(self, model: Model, dataset_reader: DatasetReader) -> None:
        super().__init__(model, dataset_reader)
        self._tokenizer = dataset_reader._tokenizer or SpacyTokenizer()

    def predict(self, sentence: str) -> JsonDict:
        return self.predict_json({"sentence" : sentence})

    @overrides
    def _json_to_instance(self, json_dict: JsonDict) -> Instance:
        sentence = json_dict["sentence"]
        tokens = self._tokenizer.tokenize(sentence)
        return self._dataset_reader.text_to_instance([str(t) for t in tokens])


@Predictor.register("universal_pos_predictor")
class UniversalPOSPredictor(Predictor):
    def __init__(self, model: Model, dataset_reader: DatasetReader) -> None:
        super().__init__(model, dataset_reader)

    def predict(self, words: List[str]) -> JsonDict:
        return self.predict_json({"words" : words})

    @overrides
    def _json_to_instance(self, json_dict: JsonDict) -> Instance:
        words = json_dict["words"]
        # This is a hack - the second argument to text_to_instance is a list of POS tags
        # that has the same length as words. We don't need it for prediction so
        # just pass words.
        return self._dataset_reader.text_to_instance(words, words)

In [34]:
predictor = SentenceClassifierPredictor(model, dataset_reader=reader)
logits = predictor.predict('This is the best movie ever!')['logits']

In [35]:
logits

[0.3762405216693878,
 -0.6694225072860718,
 -0.47617673873901367,
 1.10770583152771,
 -0.13831138610839844]

Note the predictor returns the raw output from the model, in this case 'logits'<br>
(Remember, logits are a kind of score corresponding to target labels).<br>
If you want the predicted albels themselves you need to convert it, in this case it can be done as follows/

In [37]:
label_id = np.argmax(logits)
print(model.vocab.get_token_from_index(label_id, 'labels'))

4


If this prints out a “4,” congratulations! Label “4” corresponds to “very positive,” so your sentiment analyzer just predicted that the sentence “This is the best movie ever!” is very positive, which is indeed correct.